In [ ]:
import pandas as pd
df = pd.read_parquet('metadata_files/metadata.parquet', engine='pyarrow')

# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("excel_painters_copy_paste_name.txt")
hundred_artist_names = my_utils.read_lines_as_list("100_copy_paste_names.txt")
ten_artist_names = my_utils.read_lines_as_list("10_copy_paste_names.txt")

In [ ]:
df.shape

In [ ]:
len(df["prompt"].unique())

# Exact matching

In [ ]:
import my_utils


pd.options.display.max_colwidth = 1000

painter_exact_match_list = dict()

counter = 0

for name in ten_artist_names:
    exact_matches = my_utils.exact_match_dataframe(df, name)
    
    painter_exact_match_list[name] = exact_matches.shape[0]
    #print(f'{name} exact match found {painter_exact_match_list[name]} times')
    
    counter += 1
    if counter % 1000 == 0:
        print(counter)

padding = 10

print("artists with no exact match:")
for name, occs in painter_exact_match_list.items():
    if occs == 0:
        lastname = name.split(" ")[-1]
        print(f'{name} --> lastname: {lastname}')
        lastname_matches = my_utils.exact_match_dataframe(df, lastname)
        if lastname_matches.shape[0] > 0:
            #print(lastname_matches["prompt"].head())
            for index, row in lastname_matches.iterrows():
                startindex = row['prompt'].index(lastname) -padding if row['prompt'].index(lastname) - padding > 0 else 0
                end = row['prompt'].index(lastname) + len(lastname) + padding if row['prompt'].index(lastname) + len(lastname) + padding < len(row["prompt"]) else len(row["prompt"])
                # print(row['prompt'][startindex:end])
                

with open("names_exact_match_amount_100.txt", "w") as f:
    # sort descending
    artist_occ_list = [(occs, artist) for artist, occs in painter_exact_match_list.items()]
    artist_occ_list.sort(key=lambda x: x[0], reverse=True)
    for entry in artist_occ_list:
        f.write(f'{entry[1]}: {entry[0]}\n')
            
            

In [ ]:
import my_utils

# reload the list from file
painter_exact_match_list = my_utils.read_occurences_list("names_exact_match_amount_all_artists.txt")
#print(painter_exact_match_list)

In [ ]:

import matplotlib.pyplot as plt


occs = list(painter_exact_match_list.values())
occs.sort(reverse=True)

x_data = range(len(occs))
y_data = occs

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(x_data, y_data)
ax.set_xlabel('Rank')
ax.set_ylabel('Occurences')
ax.set_title('Occurences Distribution')

plt.show()


In [ ]:
import my_utils


pd.options.display.max_colwidth = 1000

name = "greg rutkowski"
exact_matches = my_utils.exact_match_dataframe(df, name)

print(f'{name} exact match found {exact_matches.shape[0]} times')
print(f'some prompts:')
print(exact_matches.head(10)["prompt"])

## Top Artist exact matching

In [ ]:
import matplotlib.pyplot as plt

top_x = 10

artist_occ_list = [(occs, artist) for artist, occs in painter_exact_match_list.items()]
artist_occ_list.sort(key=lambda x: x[0], reverse=True)


fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(1, 1, 1) # fig.add_axes([0,0,1,1])
names = [x[1] for x in artist_occ_list[0:top_x]]
values = [x[0] for x in artist_occ_list[0:top_x]]
ax.bar(names,values)
plt.show()
# print(artist_occ_list[0:top_x])

In [ ]:
import my_utils

# to check what is currently being loaded via my_utils: (need to restart the kernel to get the changes in my_utils updated)
print(dir(my_utils))

## Exact Matching Covariance Matrix

In [ ]:
painter_names = ten_artist_names

cols = ["prompt", "image_name"]

occurences =  pd.DataFrame(columns=cols)
for name in painter_names:
    # create new column for each painter with default value 0
    occurences[painter_names] = 0

print(occurences)

cp = df.copy()

cp['painters'] = cp['prompt'].map(
        lambda p: my_utils.extract_painters_exact(p, artist_list))

for entry in cp:
    print(entry)
    occurences[entry['image_name']] = entry['image_name']
    
    for name in entry['painters']:
        occurences[name] = 1

# https://www.w3resource.com/pandas/dataframe/dataframe-cov.php
occurences.cov()

# Fuzzy Matching

In [ ]:
import time
import my_utils

starttime = time.time()

n = 1000
similarity_threshold = 0.8

df_reduced = df.head(n).copy()
assert df_reduced.shape[0] == n, f'shape is {df_reduced.shape[0]}'

artist_list = hundred_artist_names
df_reduced['painters'] = df_reduced['prompt'].map(
        lambda p: my_utils.extract_painters_fuzzy(p, artist_list, similarity_threshold))
df_reduced.to_parquet(f'{n}_entries.parquet')
df_reduced.to_csv(f'{n}_entries.csv')

duration = (time.time()-starttime)
print(f'processing took {duration/60} minutes, average {duration/n} seconds per row for {len(artist_list)} artists')

goal = 1600000
print(f'for {goal} entries it would take approximately {duration/n * goal / 60 / 60} hours')


df_reduced['painters_amount'] = df_reduced['painters'].apply(
        lambda query: len(query))
    
entries_with_painters = df_reduced.loc[df_reduced['painters_amount'] > 0, ["image_name", "prompt","user_name","painters","painters_amount"]]
entries_with_painters.to_parquet("painters_filtered.parquet")
entries_with_painters.to_csv("painters_filtered.csv")

# prognose: 933.333333333 stunden für den gesamten Datensatz (mit allen Namen aus )

# prognose: 0.11 * 1600000 /60 /60 = 48 stunden für nur die Namen aus 100_copy_paste_names.txt